In [ ]:
# 1. Imports
import lib
import torch
import torch.nn as nn
import torch.optim as optim

ModuleNotFoundError: No module named 'lib'

In [ ]:
# 2. Chargement Rice
X, y, features = lib.load_rice_dataset()
lib.plot_correlation_matrix(X, features)

NameError: name 'lib' is not defined

In [ ]:
# 3. Modèles Sklearn (Rice)
# Split manuel pour sklearn si besoin, ou utiliser cross_val
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

pipe_knn, acc = lib.train_eval_sklearn("KNN", lib.KNeighborsClassifier(n_neighbors=7), X_train, y_train, X_test, y_test)


In [ ]:
# 4. Feature Importance (Groupée)
groups = [[0, 1, 2, 3, 5], [4], [6]] # Taille, Forme, Remplissage
lib.feature_importance_permutation(pipe_knn, X_test, y_test, ["Size", "Shape", "Extent"], groups=groups)


In [ ]:
# 5. PyTorch (Rice)
train_loader, val_loader, test_loader, input_dim = lib.get_train_val_test_loaders(X, y)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Modèle binaire : output_dim=1
model = lib.GenericMLP(input_dim, output_dim=1, hidden_dims=[64])
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5)

history = lib.train_torch_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=50, device=device)
lib.plot_history(history)

In [ ]:
# 6. CovType (Multiclasse)
X_cov, y_cov = lib.load_covtype_dataset()
train_l, val_l, test_l, input_d = lib.get_train_val_test_loaders(X_cov, y_cov)

In [ ]:
# Modèle Multiclasse : output_dim=7
model_cov = lib.GenericMLP(input_d, output_dim=7, hidden_dims=[128, 64])
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_cov.parameters(), lr=0.001)

hist_cov = lib.train_torch_model(model_cov, train_l, val_l, criterion, optimizer, epochs=30, device=device)
lib.plot_history(hist_cov)